In [9]:
import pandas as pd

### Adresses

In [10]:
col_adr_i = [(0,2), (2,5), (13,20),
            (110,135), (257,262), (165,190), (160,165),
            (262,263), (263,267), (268,272)]

In [11]:
col_adr_n = ["code province", "code commune", "code unité pop",
            "entité singulière", "code postal", "nom abrégé voie","code voie",
            "type num", "début num", "fin num"]

In [12]:
adr_p02 = r"P02\TRAM.P02.D240630.G240703" # P02
adr_all = r"caj_esp_072024\TRAM.P01-52.D240630.G240703" #  P01-52

In [13]:
adr = pd.read_fwf(adr_all, colspecs=col_adr_i, header=None, encoding='ISO-8859-1', dtype=str)

In [14]:
adr.columns = col_adr_n

In [15]:
adr.head()

,code province,code commune,code unité pop,entité singulière,code postal,nom abrégé voie,code voie,type num,début num,fin num
0,01,001,0001701,ALEGRIA-DULANTZI,01240,TORRONDOA,01001,1,0001,0027
1,01,001,0001701,ALEGRIA-DULANTZI,01240,AÑUA BIDEA,01002,1,0009,0039
2,01,001,0001701,ALEGRIA-DULANTZI,01240,AÑUA BIDEA,01002,2,0004,0024
3,01,001,0001701,ALEGRIA-DULANTZI,01240,GOIKOLANDA,01003,2,0002,0010
4,01,001,0001701,ALEGRIA-DULANTZI,01240,TORREALDEA,01004,1,0005,0005


### Noms de voies

In [16]:
cols_vias_i = [(0,2), (2,5), (22,27), (27,32), 
               (32,33), (33,83), (83,108)]

In [17]:
cols_vias_n = ["code province", "code commune", "code voie", 
               "type de voie", "pos type voie", "nom de voie", "nom court"]

In [18]:
vias_p02 = r"P02\VIAS.P02.D240630.G240703" # P02
vias_all = r"caj_esp_072024\VIAS.P01-52.D240630.G240703" # P01-52

In [19]:
vias = pd.read_fwf(vias_all, colspecs=cols_vias_i, header=None, encoding='ISO-8859-1', dtype=str)

In [20]:
vias.columns = cols_vias_n

In [21]:
vias.head()

,code province,code commune,code voie,type de voie,pos type voie,nom de voie,nom court
0,01,001,01001,KALE,0,TORRONDOA,TORRONDOA
1,01,001,01002,KALE,0,AÑUA BIDEA,AÑUA BIDEA
2,01,001,01003,KALE,0,GOIKOLANDA,GOIKOLANDA
3,01,001,01004,KALE,0,TORREALDEA,TORREALDEA
4,01,001,01005,CALLE,0,NUESTRA SEÑORA DE AIALA,NUESTRA SEÑORA DE AIALA


### préparation corpus

In [22]:
adr_m_vias = pd.merge(adr, vias, on=["code province", "code commune", "code voie"])

In [23]:
adr_m_vias["fin num"] = adr_m_vias["fin num"].astype(int)
adr_m_vias["fin num"] = adr_m_vias["fin num"].astype(str)

In [24]:
adr_m_vias["adresses"] = (adr_m_vias["fin num"]+"<>"+
                         adr_m_vias["type de voie"]+" "+
                         adr_m_vias["nom de voie"]+"<>"+
                         adr_m_vias["code postal"]+"<>"+
                         adr_m_vias["entité singulière"])

In [25]:
adr_m_vias["labels"] = "num rue cp ville"

In [26]:
adr_m_vias[["fin num", "type de voie", "nom de voie", 
            "code postal", "entité singulière", "adresses", "labels"]].head()

,fin num,type de voie,nom de voie,code postal,entité singulière,adresses,labels
0,27,KALE,TORRONDOA,01240,ALEGRIA-DULANTZI,27<>KALE TORRONDOA<>01240<>ALEGRIA-DULANTZI,num rue cp ville
1,39,KALE,AÑUA BIDEA,01240,ALEGRIA-DULANTZI,39<>KALE AÑUA BIDEA<>01240<>ALEGRIA-DULANTZI,num rue cp ville
2,24,KALE,AÑUA BIDEA,01240,ALEGRIA-DULANTZI,24<>KALE AÑUA BIDEA<>01240<>ALEGRIA-DULANTZI,num rue cp ville
3,10,KALE,GOIKOLANDA,01240,ALEGRIA-DULANTZI,10<>KALE GOIKOLANDA<>01240<>ALEGRIA-DULANTZI,num rue cp ville
4,5,KALE,TORREALDEA,01240,ALEGRIA-DULANTZI,5<>KALE TORREALDEA<>01240<>ALEGRIA-DULANTZI,num rue cp ville


### sélection corpus

In [28]:
corpus_size = len(adr_m_vias)
corpus_adr = adr_m_vias["adresses"][:corpus_size].copy()
corpus_lab = adr_m_vias["labels"][:corpus_size].copy()

In [29]:
corpus_adr.head()

0     27<>KALE TORRONDOA<>01240<>ALEGRIA-DULANTZI
1    39<>KALE AÑUA BIDEA<>01240<>ALEGRIA-DULANTZI
2    24<>KALE AÑUA BIDEA<>01240<>ALEGRIA-DULANTZI
3    10<>KALE GOIKOLANDA<>01240<>ALEGRIA-DULANTZI
4     5<>KALE TORREALDEA<>01240<>ALEGRIA-DULANTZI
Name: adresses, dtype: object

In [30]:
corpus_lab.head()

0    num rue cp ville
1    num rue cp ville
2    num rue cp ville
3    num rue cp ville
4    num rue cp ville
Name: labels, dtype: object

### ajout variabilité dans les données

In [31]:
def t(x):
    # inversion du numéro de rue et du nom de rue
    x=x.split("<>")
    num, rue = x[0], x[1]
    x[0], x[1] = rue, num
    return "<>".join(x)

In [32]:
def tl(x):
    # inversion des labels "num" et "rue"
    x=x.split()
    num, rue = x[0], x[1]
    x[0], x[1] = rue, num
    return " ".join(x)

In [33]:
# application de la transformation à 10% des données
freq = int(len(corpus_adr)*0.1)
corpus_adr[:freq] = corpus_adr[:freq].transform(t)
corpus_lab[:freq] = corpus_lab[:freq].transform(tl)

In [34]:
# mélange des données
corpus = pd.DataFrame({"adresses":corpus_adr, "labels":corpus_lab})
corpus = corpus.sample(frac=1)
corpus_adr = corpus["adresses"]
corpus_lab = corpus["labels"]

In [35]:
def t2(x):
    # inversion du code postal et du nom de ville
    x=x.split("<>")
    cp, ville = x[-2], x[-1]
    x[-2], x[-1] = ville, cp
    return "<>".join(x)

In [36]:
def tl2(x):
    # inversion des labels "cp" et "ville"
    x=x.split()
    cp, ville = x[-2], x[-1]
    x[-2], x[-1] = ville, cp
    return " ".join(x)

In [37]:
# application de la transformation à 30% des données
freq = int(len(corpus_adr)*0.3)
corpus_adr[:freq] = corpus_adr[:freq].transform(t2)
corpus_lab[:freq] = corpus_lab[:freq].transform(tl2)

In [38]:
# mélange des données
corpus = pd.DataFrame({"adresses":corpus_adr, "labels":corpus_lab})
corpus = corpus.sample(frac=1)
corpus_adr = corpus["adresses"]
corpus_lab = corpus["labels"]

### extraction features

In [33]:
# Define a function to extract features for each token in an adress
def token_features(adress, i):
    token = adress[i]
    features = {
        'token': token,
#         'is_first': i == 0, #if the token is the first token
#         'is_last': i == len(adress) - 1, #if the token is the last token
        'token_length': len(token),
#         #prefix of the token
#         'prefix-1': token[0], 
#         'prefix-2': token[:2],
#         'prefix-3': token[:3],
#         #suffix of the token
#         'suffix-1': token[-1],
#         'suffix-2': token[-2:],
#         'suffix-3': token[-3:],
        #extracting previous token
        'prev_token': '' if i == 0 else adress[i-1][0],
        #extracting next token
        'next_token': '' if i == len(adress)-1 else adress[i+1],
        'has_hyphen': '-' in token, #if token has hypen
        'is_numeric': token.isdigit(), #if token is in numeric
    }
    return features

In [34]:
def process_adress(adress, labels):
    X_adress = []
    y_adress = []
    labels = labels.split()
    elts = adress.split("<>")
    for i, elt in enumerate(elts):
        tokens = elt.split()
        for j, t in enumerate(tokens):
            X_adress.append(token_features(adress.replace("<>", " ").split(), i+j))
            y_adress.append(labels[i])
    return X_adress, y_adress

In [35]:
# Extract features for each sentence in the corpus
X = []
y = []
for i in corpus.index:
    X_adress, y_adress = process_adress(corpus_adr[i], corpus_lab[i])
    X.append(X_adress)
    y.append(y_adress)

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train = X[:split]
y_train = y[:split]
X_test = X[split:]
y_test = y[split:]

### entraînement / évaluation modèle

In [93]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [449]:
# Train a CRF model on the training data
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

# Make predictions on the test data and evaluate the performance
y_pred = crf.predict(X_test)

print(metrics.flat_accuracy_score(y_test, y_pred))

loading training data to CRFsuite: 100%|██████████████████████████████████| 1152152/1152152 [00:51<00:00, 22252.66it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 532684
Seconds required: 11.471

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=19.12 loss=9532311.21 active=496574 feature_norm=0.25
Iter 2   time=4.55  loss=9259344.86 active=498613 feature_norm=0.27
Iter 3   time=4.54  loss=8985506.42 active=472924 feature_norm=0.35
Iter 4   time=4.50  loss=7519145.30 active=480973 feature_norm=1.08
Iter 5   time=4.45  loss=4710888.87 active=505821 feature_norm=2.83
Iter 6   time=4.42  loss=4403321.12 active=498123 feature_norm=4.19
Iter 7   time=4.42  loss=3801067.67 active=527615 feature_norm=4.67
Iter 8   time=4.47  loss=3583521.73 active=524755 feature_norm=5.27
Iter 9   time=4.50  loss=3479718.33 active=516171 fea

### predict function

In [408]:
from pprint import pprint
def predict(adress):
    print(adress)
    X_adress = []
    tokens = adress.split()
    for i in range(len(tokens)):
        X_adress.append(word_features(tokens, i))
    tags = crf.predict([X_adress]).tolist()[0]
    pprint(list(zip(tokens, tags)))

In [543]:
predict(corpus_adr.iloc[-537].replace("<>", " "))

29 CALLE CANTERAS (LAS) 28411 MORALZARZAL
[('29', 'num'),
 ('CALLE', 'rue'),
 ('CANTERAS', 'rue'),
 ('(LAS)', 'rue'),
 ('28411', 'ville'),
 ('MORALZARZAL', 'cp')]


In [39]:
# y_test[-537]

### Deep learning

Le problème de parsing d'adresses postales peut se rapprocher d'un problème de reconnaissance d'entités nommées (NER). Il s'agit de classifier une séquence de tokens en sachant qu'une catégorie d'une adresse postale - le nom de la rue, par exemple - peut s'étendre sur plusieurs tokens. Il en est de même pour les entités nommées. Par exemple un nom de personne s'étendra souvent sur 2 tokens ou plus (nom et prénom).  
  
Spacy est une libairie performante pour le problème de reconnaissance des entités nommées. C'est une librairie "high-level" basée sur des réseaux de neurones. C'est cette solution que j'ai choisi pour la partie deep learning de résolution du usecase. 

### Librairie Spacy

L'architecture est composée d'une couche d'embeddings suivie de réseaux convolutifs pour capturer le contexte. L'output sert à prédire la prochaine action à effectuer dans un algorithme de parsing séquentiel de type shift-reduce.  
  
L'inconvénient de cette librairie est son opacité. Il semble par exemple difficile d'accéder aux vecteurs contextuels des mots. De même, il est difficile d'accéder aux features utilisés à l'étape finale de prédiction. Ceux-ci sont malgré tout présentés dans une vidéo du fondateur de Spacy et sont les suivants:  
mot courant, mot précédant, mot suivant; premier et dernier mot de l'entité précédente, dernier mot de l'entité encore avant  
Il n'est pas clair si ces features sont personnalisables ou non, même si cela semble probable étant donné que cette flexibilité est mise en avant comme un des atouts de leur solution.  
  
Concernant les embeddings, il est possible de charger des embeddings pré-entraînés. De même, il est possible de changer leur architecture à base de CNN par une architecture de type transformer.

### Comparaison CRF

Les résultats obtenus dans les mêmes conditions qu'avec les CRF plus haut sont largement supérieurs. Là où les CRF atteignent un f1-score de 0.86, le module NER de Spacy atteint un score au-delà de 0.99. Cela peut également laisser supposer un overfitting qui demande à être contrôlé en ajoutant de la variabilité dans les données de développement. 

Le coût computationnel est bien entendu supérieur. Là où le CRF a pu réaliser 100 époques en 10 minutes, le module NER de Spacy a pris plusieurs heures pour réaliser une seule époque. A noter néanmoins la possibilité de basculer une partie des calculs sur le GPU pour améliorer les performances. 

### Préparation des données

In [39]:
from tqdm import tqdm

In [40]:
def format_data(raw_adress, raw_labels):
    adress_parts = raw_adress.split("<>")
    labels = raw_labels.split()
    labels_new = {"entities":[]}
    left_bound = 0
    for i, ap in enumerate(adress_parts):
        labels_new["entities"].append((left_bound, left_bound+len(ap), labels[i]))
        left_bound += len(ap)+1
    return (raw_adress.replace("<>", " "), labels_new)

In [59]:
data = []
for i in tqdm(range(len(corpus_adr))):
    data.append(format_data(corpus_adr.iloc[i], corpus_lab.iloc[i]))

100%|█████████████████████████████████████████████████████████████████████| 1440190/1440190 [00:43<00:00, 32998.48it/s]


In [60]:
split = int(len(data)*0.8)
train_data = data[:split]
test_data = data[split:]

### Initialisation modèle

In [62]:
import spacy

# spacy.prefer_gpu()

# Créer un modèle vierge pour l'espagnol
nlp = spacy.blank("es")

# Ajouter un composant NER au pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")

# Possibilité d'utiliser un modèle pré-entrainé :
# nlp = spacy.load("es_core_news_sm")
# ner = nlp.get_pipe("ner")

# Ajouter les étiquettes d'entités
for _, annotations in tqdm(train_data):
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

100%|█████████████████████████████████████████████████████████████████████| 1152152/1152152 [00:24<00:00, 47112.00it/s]


In [63]:
from spacy.training.example import Example

for i in tqdm(range(len(train_data[:100000]))):
    text, labels = train_data[i]
    doc = nlp.make_doc(text)
    train_data[i] = Example.from_dict(doc, labels)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:28<00:00, 3571.34it/s]


### Entraînement

In [64]:
from spacy.util import minibatch, compounding
import random

# spacy.require_gpu()

# Désactiver les autres composants du pipeline pendant l'entraînement
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # Désactiver les pipes sauf 'ner'
    optimizer = nlp.begin_training()
    
    # Boucles sur plusieurs itérations (époques)
    for itn in range(1):
#         random.shuffle(train_data)
        losses = {}
        
        # Diviser les données en mini-batchs et entraîner
        batches = minibatch(train_data[:100000], size=compounding(4.0, 32.0, 1.001))
        for batch in tqdm(batches):
#             texts, annotations = zip(*batch)
            nlp.update(batch, drop=0.5, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")

1469it [01:29, 16.47it/s]


KeyboardInterrupt: 

In [45]:
nlp.to_disk("spacy_model")

### Evaluation

In [48]:
for i in tqdm(range(len(test_data))):
    text, labels = test_data[i]
    doc = nlp.make_doc(text)
    test_data[i] = Example.from_dict(doc, labels)

100%|████████████████████████████████████████████████████████████████████████| 288038/288038 [01:38<00:00, 2920.16it/s]


In [70]:
for i in range(5):
    doc = nlp(data[-i][0]); print()

    # Parcourir les entités détectées
    for ent in doc.ents:
        print(ent.text, ent.label_)


13 num
C LLUNA rue
08191 cp
RUBI ville

13 num
CALLE ALEJANDRO CASONA rue
33940 cp
ENTREGU (L')/ENTREGO (EL) ville

15 num
CALLE MEDICO TABERNER rue
QUARTELL ville
46510 cp

6 num
CALLE FRAY FELICIANO DE VENTOSA rue
47232 cp
VENTOSA DE LA CUESTA ville

CALLE MANCHA (LA) rue
31 num
PETRER ville
03610 cp


In [62]:
nlp.evaluate(test_data)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.9968264847115164,
 'ents_r': 0.9972703254431707,
 'ents_f': 0.9970483556829024,
 'ents_per_type': {'num': {'p': 0.9999027865539917,
   'r': 0.9998611294343107,
   'f': 0.9998819575602712},
  'rue': {'p': 0.992032467261946,
   'r': 0.9920531318784327,
   'f': 0.9920427994625766},
  'cp': {'p': 0.9998299124917126, 'r': 1.0, 'f': 0.9999149490128011},
  'ville': {'p': 0.9955425846077058,
   'r': 0.9971670404599393,
   'f': 0.9963541504060386}},
 'speed': 15846.461693305087}

In [77]:
# Créer un modèle vierge pour l'espagnol
nlp_test = spacy.blank("es")

# Ajouter un composant NER au pipeline
if "ner" not in nlp_test.pipe_names:
    ner_test = nlp_test.add_pipe("ner")

# Ajouter les étiquettes d'entités
for _, annotations in tqdm(data[:split]):
    for ent in annotations.get("entities"):
        ner_test.add_label(ent[2])

In [80]:
# Désactiver les autres composants du pipeline pendant l'entraînement
other_pipes = [pipe for pipe in nlp_test.pipe_names if pipe != "ner"]
with nlp_test.disable_pipes(*other_pipes):  # Désactiver les pipes sauf 'ner'
    optimizer = nlp_test.begin_training()
    
    # Boucles sur plusieurs itérations (époques)
    for itn in range(1):
        random.shuffle(train_data)
        losses = {}
        
        # Diviser les données en mini-batchs et entraîner
        batches = minibatch(train_data[:5], size=compounding(4.0, 32.0, 1.001))
        for batch in tqdm(batches):
            nlp_test.update(batch, drop=0.5, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")

2it [00:00,  6.70it/s]

Iteration 0, Losses: {'ner': 33.0120667219162}


In [81]:
nlp_test.evaluate(test_data[:5000])

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.08471391972672929,
 'ents_r': 0.0248,
 'ents_f': 0.03836782053761361,
 'ents_per_type': {'num': {'p': 0.21593447505584512,
   'r': 0.058,
   'f': 0.09143938199590099},
  'ville': {'p': 0.06666666666666667, 'r': 0.0006, 'f': 0.0011892963330029731},
  'rue': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'cp': {'p': 0.12258454106280194, 'r': 0.0406, 'f': 0.060997596153846145}},
 'speed': 12272.06301893324}

In [82]:
for i in range(5):
    doc = nlp_test(data[-i][0]); print()

    # Parcourir les entités détectées
    for ent in doc.ents:
        print(ent.text, ent.label_)


08191 num

CASONA 33940 ENTREGU (L')/ENTREGO (EL) rue

QUARTELL num

VENTOSA 47232 VENTOSA DE LA CUESTA rue

31 num


### Test robustness

 - - -
### Brouillon

lignes exclues

In [252]:
# Jointure externe pour voir les lignes exclues
outer_merged = pd.merge(adr, vias, on=["code province", "code commune", "code voie"], how='outer', indicator=True)

In [253]:
# Filtrer les lignes qui ne sont présentes que dans l'un des DataFrames
excluded_rows = outer_merged[outer_merged['_merge'] != 'both']
excluded_rows

,code province,code commune,code unité pop,entité singulière,code postal,nom abrégé voie,code voie,type num,début num,fin num,type de voie,pos type voie,nom de voie,nom court,_merge
1,02,001,NaN,NaN,NaN,NaN,00009,NaN,NaN,NaN,CTRA,0,CARRETERA DE JORQUERA,CARRETERA DE JORQUERA,right_only
31,02,001,NaN,NaN,NaN,NaN,00168,NaN,NaN,NaN,CALLE,0,EXTRAMUROS,EXTRAMUROS,right_only
66,02,001,NaN,NaN,NaN,NaN,00341,NaN,NaN,NaN,CALLE,0,FRONTON,FRONTON,right_only
79,02,001,NaN,NaN,NaN,NaN,00363,NaN,NaN,NaN,CALLE,0,PONIENTE,PONIENTE,right_only
81,02,002,NaN,NaN,NaN,NaN,00022,NaN,NaN,NaN,.,0,CEMENTERIO,CEMENTERIO,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18272,02,901,NaN,NaN,NaN,NaN,00067,NaN,NaN,NaN,CLLON,0,TRASERAS,TRASERAS,right_only
18279,02,901,NaN,NaN,NaN,NaN,00075,NaN,NaN,NaN,CALLE,0,POZO BUENO,POZO BUENO,right_only
18293,02,901,NaN,NaN,NaN,NaN,00084,NaN,NaN,NaN,CALLE,0,ORTEGA Y GASSET,ORTEGA Y GASSET,right_only
18295,02,901,NaN,NaN,NaN,NaN,00086,NaN,NaN,NaN,CALLE,0,LEON FELIPE,LEON FELIPE,right_only


In [259]:
print("lignes perdues:", len(excluded_rows),
      "\nvias:", len(excluded_rows[excluded_rows["_merge"]=="right_only"]), 
      "\nadr:", len(excluded_rows[excluded_rows["_merge"]=="left_only"]))

lignes perdues: 2487 
vias: 2035 
adr: 452


In [280]:
len(adr) - len(adr[adr.duplicated(subset=["code province", "code commune", "code voie"])])

8448

In [279]:
# (len(vias) - len(excluded_rows[excluded_rows["_merge"]=="right_only"]))

exemple

In [218]:
adr[(adr["code province"]=="01") & (adr["code commune"]=="001") & (adr["code voie"]=="01024")]

,code province,code commune,code unité pop,entité singulère,code postal,nom abrégé voie,code voie,type num,début num,fin num


In [219]:
vias[(vias["code province"]=="01") & (vias["code commune"]=="001") & (vias["code voie"]=="01024")]

,code province,code commune,code voie,type de voie,pos type voie,nom de voie,nom court
23,01,001,01024,KALE,0,ANUNZARGARAI,ANUNZARGARAI


In [45]:
vias[vias["code voie"]=="01001"].head()

,code province,code commune,code voie,type de voie,pos type voie,nom de voie,nom court
0,01,001,01001,KALE,0,TORRONDOA,TORRONDOA
185,01,004,01001,CALLE,0,GOIENKALE,GOIENKALE
823,01,018,01001,CALLE,0,GOROSTIZA,GOROSTIZA
941,01,019,01001,CALLE,0,SAN MARTIN,SAN MARTIN
1641,01,042,01001,BARRO,0,IRABIEN,IRABIEN


In [53]:
vias.loc[:,["code voie", "code commune"]].head()

,code voie,code commune
0,01001,001
1,01002,001
2,01003,001
3,01004,001
4,01005,001


Objectif : enrichir adr avec les "types de voie" de vias  
adr ne doit pas perdre d'information

## Clé primaire

Recherche d'une clé primaire pour joindre les dataframes adr et vias

Est-ce que le set d'attributs est une clé primaire?

In [146]:
not adr[["code province", "code commune", "code voie"]].duplicated().any()

False

In [178]:
not adr[["code province", "code commune", "code voie", "type num"]][:1000].duplicated().any()

False

In [262]:
not vias[["code province", "code commune", "code voie", "type de voie"]].duplicated().any()

True

Affiche toutes les lignes qui sont des duplicats d'autres lignes sur les attributs spécifiés

In [264]:
len(adr[adr.duplicated(subset=["code province", "code commune", "code voie"])])

7820

In [ ]:
adr[
    (adr["code province"]=="01") 
    & (adr["code commune"]=="002") 
    & (adr["code voie"]=="00090") 
#     & (adr["type num"]=="1")
   ]

exemple gpt

In [224]:
df1 = pd.DataFrame({'key': [1, 2, 3], 'val1': ['A', 'B', 'C']})
df1

,key,val1
0,1,A
1,2,B
2,3,C


In [225]:
df2 = pd.DataFrame({'key': [2, 3, 4], 'val2': ['X', 'Y', 'Z']})
df2

,key,val2
0,2,X
1,3,Y
2,4,Z


In [226]:
pd.merge(df1, df2, on='key', how='inner')
# merged_df ne contiendra que les lignes avec les clés 2 et 3, car 1 et 4 ne sont pas dans les deux DataFrames.

,key,val1,val2
0,2,B,X
1,3,C,Y


In [211]:
vias[vias.duplicated(subset=["code voie", "code commune"])]

,code province,code commune,code voie,type de voie,pos type voie,nom de voie,nom court
4124,02,002,00075,.,0,ESTACION DE SERVICIO,ESTACION DE SERVICIO
4128,02,002,00090,.,0,GOBERNADORA (LA),GOBERNADORA (LA)
4131,02,002,00115,.,0,PIEDRAS LISAS (LAS),PIEDRAS LISAS (LAS)
4132,02,002,00120,.,0,TALLERES AYORA,TALLERES AYORA
4146,02,002,00265,CALLE,0,HIGUERAS (LAS),HIGUERAS (LAS)
...,...,...,...,...,...,...,...
903576,52,001,04330,CALLE,0,PANAMA,PANAMA
903693,52,001,05080,CALLE,0,"SAMANIEGO, TENIENTE","SAMANIEGO, TENIENTE"
903814,52,001,06010,CALLE,0,ZAMORA,ZAMORA
903815,52,001,06015,LUGAR,0,ZAWIYA AL ALAWIYA,ZAWIYA AL ALAWIYA


Affiche les lignes qui ont des valeurs spécifiques à des attributs spécifiques

In [189]:
vias[(vias["code voie"]=="00075") & (vias["code commune"]=="002")]

,code province,code commune,code voie,type de voie,pos type voie,nom de voie,nom court
84,01,002,00075,CALLE,0,BORIÑAUR KALEA,BORIÑAUR KALEA
4124,02,002,00075,.,0,ESTACION DE SERVICIO,ESTACION DE SERVICIO
81684,06,002,00075,CALLE,0,PINO,PINO
119623,08,002,00075,.,0,CAN VILA,CAN VILA
274667,14,002,00075,CALLE,0,JARA (LA),JARA (LA)
290941,15,002,00075,RUA,0,PALMEIRAS (DAS),PALMEIRAS (DAS)
556606,30,002,00075,PLAZA,0,ARANAGA Y GOROSTIZA,ARANAGA Y GOROSTIZA
612713,33,002,00075,BARRO,0,SANTA BARBARA,SANTA BARBARA


### Data prep (spacy)

In [41]:
def get_token_labels(raw_adress, raw_labels):
    
    # preparation
    token_labels = []
    labels = raw_labels.split()
    
    # iteration over adress parts
    for i, elt in enumerate(raw_adress.split("<>")):
        nb_words = len(elt.split())
        
        # adding right labels for each word
        for _ in range(nb_words):
            token_labels.append(labels[i])
            
    return token_labels

In [42]:
get_token_labels(corpus_adr.iloc[3], corpus_lab.iloc[3])

['num', 'rue', 'rue', 'cp', 'ville']

In [43]:
def get_token_boundaries(raw_adress):
    adress = raw_adress.replace("<>", " ")
    left_bound = 0
    boundaries = []
    for i in range(len(adress)):
        if adress[i]==" ":
            boundaries.append([left_bound, i])
            left_bound = i+1
    boundaries.append([left_bound, len(adress)])
    return boundaries

In [44]:
get_token_boundaries(corpus_adr.iloc[3])

[[0, 2], [3, 8], [9, 17], [18, 23], [24, 30]]

### Types de voies

In [205]:
cols_tipos_i = [(0,5), (5,10), (10,35)]

In [206]:
tipos = pd.read_fwf(r"C:\Users\vrivi\Downloads\TipoVias", 
                 colspecs=cols_tipos_i, header=None, encoding='ISO-8859-1', dtype=str)

In [207]:
tipos.head()

,0,1,2
0,ACCE,ACCE,ACCES
1,ACCE,ACCES,ACCES
2,ACCE,SARBI,ACCES
3,ACCES,ACCE,ACCESO
4,ACCES,ACCES,ACCESO


In [208]:
len(tipos)

632

In [219]:
tipos[2].str.len().value_counts().sort_index()

2
3      13
4      52
5     118
6     126
7     120
8     102
9      43
10     35
11      9
12     10
13      3
15      1
Name: count, dtype: int64

In [225]:
tipos[2][tipos[2].str.len()==10][:5]

95     BARREDUELA
107    CMNO HONDO
108    CMNO NUEVO
109    CMNO VIEJO
135    OCANTIÃO
Name: 2, dtype: object

In [214]:
tipos[2].str.len().mean()

6.738924050632911

In [ ]:
adr_m_vias.head()